In [0]:
sc

<SparkContext master=spark://10.139.64.4:7077 appName=Databricks Shell>

In [0]:
spark

In [0]:
df = spark.read.csv("/Volumes/azure_workspacedatabricks/default/sales/raw_flight_data.csv",header=True,inferSchema=True)

In [0]:
df.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



In [0]:
from pyspark.sql.functions import *

In [0]:
null_col = df.select([count(when(isnull(col), col)).alias(col) for col in df.columns])
null_col.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         0|        0|      0|              0|            0|   27444|   29033|
+----------+---------+-------+---------------+-------------+--------+--------+



In [0]:
df.dropna()

DataFrame[DayofMonth: int, DayOfWeek: int, Carrier: string, OriginAirportID: int, DestAirportID: int, DepDelay: int, ArrDelay: int]

In [0]:
df = df.withColumn("OriginAirportID",col("OriginAirportID").cast("long"))
df = df.withColumn("DestAirportID",col("DestAirportID").cast("long"))

In [0]:
df = df.dropDuplicates()
df.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        20|        6|     DL|          10397|        12953|      -4|     -25|
|        18|        4|     DL|          10397|        11298|      -3|      44|
|        19|        5|     DL|          10397|        11292|      -3|     -13|
|        19|        5|     DL|          14679|        14869|       1|       8|
|        16|        2|     DL|          10397|        14524|       4|      -2|
|        17|        3|     DL|          13487|        13342|      -5|     -16|
|        18|        4|     DL|          10397|        10792|      22|       3|
|        18|        4|     DL|          10397|        14492|       2|      -9|
|        18|        4|     DL|          10397|        11433|       0|      14|
|        20|        6|     DL|          15304|      

In [0]:
df.select(round(mean(col('ArrDelay'))).alias('Mean_ArrDelay')).show()
df.select(round(stddev(col('DepDelay'))).alias('StdDev_DepDelay')).show()
median = df.approxQuantile('ArrDelay', [0.5],0.001)
print(f"Median_ArrDelay: {median}")

+-------------+
|Mean_ArrDelay|
+-------------+
|          7.0|
+-------------+

+---------------+
|StdDev_DepDelay|
+---------------+
|           36.0|
+---------------+

Median_ArrDelay: [-3.0]


In [0]:
df.select(['OriginAirportID']).distinct().count()

70

In [0]:
df.filter((df.DayOfWeek > 4) & (df.Carrier == "DL")).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        20|        6|     DL|          10397|        12953|      -4|     -25|
|        19|        5|     DL|          10397|        11292|      -3|     -13|
|        19|        5|     DL|          14679|        14869|       1|       8|
|        20|        6|     DL|          15304|        12478|      60|      56|
|        20|        6|     DL|          11433|        10721|      -3|     -12|
|        20|        6|     DL|          10397|        14683|      -4|       6|
|        20|        6|     DL|          13198|        13487|      -8|     -12|
|        21|        7|     DL|          12892|        14869|      -5|     -12|
|        21|        7|     DL|          11278|        11433|      -5|      -9|
|        21|        7|     DL|          14747|      

In [0]:
airports_df = spark.read.csv("/Volumes/azure_workspacedatabricks/default/sales/airports.csv",header=True,inferSchema=True)

In [0]:
newairportDF = df.join(airports_df,df.OriginAirportID == airports_df.airport_id)
newairportDF.show()

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|          city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|        20|        6|     DL|          10397|        12953|      -4|     -25|     10397|       Atlanta|   GA|Hartsfield-Jackso...|
|        18|        4|     DL|          10397|        11298|      -3|      44|     10397|       Atlanta|   GA|Hartsfield-Jackso...|
|        19|        5|     DL|          10397|        11292|      -3|     -13|     10397|       Atlanta|   GA|Hartsfield-Jackso...|
|        19|        5|     DL|          14679|        14869|       1|       8|     14679|     San Diego|   CA|San Diego Interna...|
|        16|        2|     DL|          10397|        14524|       4|      -